In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
tqdm.pandas()
pd.options.display.max_columns = None

# 1. Data load and optimize

In [2]:
games = pd.read_csv('../data/games/games.csv',\
        usecols=['game_id','date', 'home_club_id', 'away_club_id','home_club_goals', 'away_club_goals'],\
        dtype = {'game_id' : np.int32, 'home_club_id' : np.int32, 'away_club_id' : np.int32,\
                'home_club_goals' : np.int8, 'away_club_goals' : np.int8})

appearance = pd.read_csv('../data/games/appearance.csv', \
        usecols=['game_id','player_club_id','yellow_cards','red_cards'],\
                dtype={'game_id':np.float32,'player_club_id':np.int32,'yellow_cards':np.int8,'red_cards':np.int8})


In [127]:
train_df = pd.read_csv("data/training_data/train_final.csv", index_col = 0)
train_df = train_df.astype(dtype={x:np.float16 for x in train_df.columns[1:-1]})


In [4]:
games.date = pd.to_datetime(games.date)

In [5]:
appearance = appearance.drop(appearance[appearance.game_id.isnull()].index).reset_index(drop=True)
appearance = appearance.astype(dtype={'game_id':np.int32})

# 2. contruire des datasets de l'historique de home et away.
##### on ajoutera dans le dataset train_df deux listes qui donneront les indices des lignes à aller chercher dans lesdits datasets pour entrainer le LSTM

ATTENTION: les parties de l'historique sont classée de la partie la plus récente à la partie la plus ancienne. C'est pas volontaire mais c'est comme c'est.

In [68]:
def get_home_history(game_id):

    N = 10
    target_game = games.loc[games.game_id == game_id,:]
    club_id = target_game['home_club_id'].values[0]

    N_last_games = games.loc[((games.home_club_id == club_id) | (games.away_club_id == club_id))\
            & (games.date < target_game['date'].values[0]) ,:].sort_values('date',ascending=False)[:N]
    
    if N_last_games.shape[0]==0:
        return None

    else:
        N_last_games[['H_goals','H_goals_conceded','H_HorA']] = N_last_games.apply(lambda x: \
        pd.Series([x['home_club_goals'], x['away_club_goals'],0]) if x['home_club_id']==club_id else pd.Series([x['away_club_goals'], x['home_club_goals'],1])\
        , axis = 1)

        cards = appearance.loc[(appearance.game_id.isin(N_last_games.game_id.values)) \
                & (appearance.player_club_id == club_id),\
                    ['game_id','yellow_cards','red_cards']].groupby('game_id').sum().reset_index()\
                        .rename(columns={'yellow_cards': 'H_yellow_cards' ,'red_cards' : 'H_red_cards'})

        N_last_games.drop(columns = ['date','home_club_id','away_club_id','home_club_goals','away_club_goals'],inplace = True)
        res = pd.merge(N_last_games,cards,on='game_id')
        res.game_id= game_id
        return res

In [121]:
game_id = game_ids[500]
target_game = games.loc[games.game_id == game_id,:]
club_id = target_game['home_club_id'].values[0]

N_last_games = games.loc[((games.home_club_id == club_id) | (games.away_club_id == club_id))\
        & (games.date < target_game['date'].values[0]) ,:].sort_values('date',ascending=False)[:N]

N_last_games[['H_goals','H_goals_conceded','H_HorA']] = N_last_games.apply(lambda x: \
pd.Series([x['home_club_goals'], x['away_club_goals'],0]) if x['home_club_id']==club_id else pd.Series([x['away_club_goals'], x['home_club_goals'],1])\
, axis = 1)
N_last_games

,game_id,date,home_club_id,away_club_id,home_club_goals,away_club_goals,H_goals,H_goals_conceded,H_HorA
2290,2460802,2015-02-07,60,16,0,3,3,0,1
2347,2460789,2015-02-04,16,167,0,1,0,1,0
2355,2460780,2015-01-31,15,16,0,0,0,0,1
2416,2460776,2014-12-20,86,16,2,1,1,2,1
2425,2460762,2014-12-17,16,82,2,2,2,2,0
2434,2460758,2014-12-13,44,16,1,0,0,1,1
5591,2495365,2014-12-09,16,58,1,1,1,1,0
2487,2460744,2014-12-05,16,533,1,0,1,0,0
2488,2460741,2014-11-30,24,16,2,0,0,2,1
5592,2495356,2014-11-26,11,16,2,0,0,2,1


In [69]:
def get_away_history(game_id):

    N = 10
    target_game = games.loc[games.game_id == game_id,:]
    club_id = target_game['away_club_id'].values[0]

    N_last_games = games.loc[((games.home_club_id == club_id) | (games.away_club_id == club_id))\
            & (games.date < target_game['date'].values[0]) ,:].sort_values('date',ascending=False)[:N]
    
    if N_last_games.shape[0]==0:
        return None

    else:
        N_last_games[['A_goals','A_goals_conceded','A_HorA']] = N_last_games.apply(lambda x: \
        pd.Series([x['home_club_goals'], x['away_club_goals'],0]) if x['home_club_id']==club_id else pd.Series([x['away_club_goals'], x['home_club_goals'],1])\
        , axis = 1)

        cards = appearance.loc[(appearance.game_id.isin(N_last_games.game_id.values)) \
                & (appearance.player_club_id == club_id),\
                    ['game_id','yellow_cards','red_cards']].groupby('game_id').sum().reset_index()\
                        .rename(columns={'yellow_cards': 'A_yellow_cards' ,'red_cards' : 'A_red_cards'})

        N_last_games.drop(columns = ['date','home_club_id','away_club_id','home_club_goals','away_club_goals'],inplace = True)
        res = pd.merge(N_last_games,cards,on='game_id')
        res.game_id= game_id
        return res

In [62]:
game_ids = train_df.game_id.astype(np.int32).copy()

In [70]:
home_history = game_ids.progress_apply(get_home_history)

100%|██████████| 29545/29545 [13:08<00:00, 37.46it/s]


In [73]:
home_history = pd.concat(home_history.values)

In [77]:
home_history = home_history.reset_index(drop = True).reset_index().rename(columns={'index':'home_id'})

In [93]:
home_history.to_csv('data/train_LSTM/home_history.csv')

In [79]:
away_history = game_ids.progress_apply(get_away_history)
away_history = pd.concat(away_history.values)
away_history = away_history.reset_index(drop = True).reset_index().rename(columns={'index':'away_id'})
away_history.to_csv('data/train_LSTM/away_history.csv')

100%|██████████| 29545/29545 [13:01<00:00, 37.80it/s]


In [85]:
away_history.loc[away_history.game_id==3710372,'away_id'].unique()

array([287526, 287527, 287528, 287529, 287530, 287531, 287532, 287533,
       287534, 287535])

In [94]:
home_history

,home_id,game_id,H_goals,H_goals_conceded,H_HorA,H_yellow_cards,H_red_cards
0,0,2459586,1,0,0,3,0
1,1,2459593,0,1,1,1,0
2,2,2459595,1,0,1,5,0
3,3,2459595,1,3,1,3,0
4,4,2459595,1,2,0,2,0
...,...,...,...,...,...,...,...
287659,287659,3710372,3,4,0,2,0
287660,287660,3710372,2,2,0,2,0
287661,287661,3710372,1,0,1,4,0
287662,287662,3710372,3,2,1,8,0


In [100]:
x =train_df.game_id.values[11]
print(home_history.loc[home_history.game_id == x,'home_id'].unique() )
print(away_history.loc[away_history.game_id == x,'away_id'].unique())

[]
[4 5]


In [129]:
train_df['home_history'] = game_ids.progress_apply(lambda x : home_history.loc[home_history.game_id == x,'home_id'].unique())

100%|██████████| 29545/29545 [00:15<00:00, 1881.08it/s]
/var/folders/fw/vk6b0tfx7kvg_95mz0b5mts80000gn/T/ipykernel_11308/1431477703.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  train_df['home_history'] = game_ids.progress_apply(lambda x : home_history.loc[home_history.game_id == x,'home_id'].unique())


In [130]:
train_df['away_history'] = game_ids.progress_apply(lambda x : away_history.loc[away_history.game_id == x,'away_id'].unique())

100%|██████████| 29545/29545 [00:15<00:00, 1864.44it/s]
/var/folders/fw/vk6b0tfx7kvg_95mz0b5mts80000gn/T/ipykernel_11308/1015295018.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  train_df['away_history'] = game_ids.progress_apply(lambda x : away_history.loc[away_history.game_id == x,'away_id'].unique())


In [132]:
train_df = train_df.loc[:,np.concatenate((['game_id', 'home_history', 'away_history'],train_df.columns[1:-2]))]

In [134]:
train_df.to_csv('data/train_LSTM/train_LSTM.csv')

In [137]:
test= pd.read_csv('data/train_LSTM/train_LSTM.csv', index_col=0)

In [140]:
test.home_history

0                                                       []
1                                                       []
2                                                       []
3                                                      [0]
4                                                       []
                               ...                        
29540    [287614 287615 287616 287617 287618 287619 287...
29541    [287624 287625 287626 287627 287628 287629 287...
29542    [287634 287635 287636 287637 287638 287639 287...
29543    [287644 287645 287646 287647 287648 287649 287...
29544    [287654 287655 287656 287657 287658 287659 287...
Name: home_history, Length: 29545, dtype: object

In [147]:
import ast # ast is part of python standard's library. 
import re
def transform_int_column(x):
    if x[0] == r'[':
        res = re.sub(r'(\d)\s+(\d)',r'\1 , \2',x.replace('\n',''))
        res = re.sub(r'(\d)\s+(\d)',r'\1 , \2',res.replace('\n',''))
        res = ast.literal_eval(res) 
        res = np.asarray([int(y) for y in res])
    else:
        res = np.asarray([])
    return res


In [149]:
test.home_history.apply(transform_int_column)

array([287634, 287635, 287636, 287637, 287638, 287639, 287640, 287641,
       287642, 287643])

## Train dataframe only for training the LSTM, using all of games.csv

In [154]:
home_history_full = games.game_id.progress_apply(get_home_history)

100%|██████████| 42592/42592 [18:36<00:00, 38.16it/s]


In [155]:
home_history_full = pd.concat(home_history_full.values)
home_history_full = home_history_full.reset_index(drop = True).reset_index().rename(columns={'index':'home_id'})
home_history_full.to_csv('data/train_LSTM/home_history_full.csv')

In [157]:
away_history_full = games.game_id.progress_apply(get_away_history)
away_history_full = pd.concat(away_history_full.values)
away_history_full = away_history_full.reset_index(drop = True).reset_index().rename(columns={'index':'away_id'})
away_history_full.to_csv('data/train_LSTM/away_history_full.csv')

100%|██████████| 42592/42592 [18:20<00:00, 38.69it/s]


In [165]:
away_history_full

,away_id,game_id,A_goals,A_goals_conceded,A_HorA,A_yellow_cards,A_red_cards
0,0,2484338,0,0,0,2,0
1,1,2484338,1,0,1,2,0
2,2,2459774,3,1,0,2,0
3,3,2459774,0,3,1,3,0
4,4,2459774,1,1,1,3,0
...,...,...,...,...,...,...,...
359165,359165,3612889,3,3,0,1,0
359166,359166,3612889,2,1,0,1,0
359167,359167,3612865,1,1,0,0,0
359168,359168,3612865,1,1,1,5,0


### on rajoute encore des features? allez! L'historique des rencontres entre les 2 équipes dont on cherche à prédire le résultat.

In [170]:
game_id = game_ids[100]
N = 5
target_game = games.loc[games.game_id == game_id,:]
club_home_id = target_game['home_club_id'].values[0]
club_away_id = target_game['away_club_id'].values[0]

N_last_games = games.loc[((games.home_club_id == club_home_id) | (games.away_club_id == club_home_id))\
    & ((games.home_club_id == club_away_id) | (games.away_club_id == club_away_id))\
    & (games.date < target_game['date'].values[0]) ,:].sort_values('date',ascending=False)[:N]
N_last_games

,game_id,date,home_club_id,away_club_id,home_club_goals,away_club_goals


In [171]:
def get_game_history(game_id):

    N = 5
    target_game = games.loc[games.game_id == game_id,:]
    club_home_id = target_game['home_club_id'].values[0]
    club_away_id = target_game['away_club_id'].values[0]

    N_last_games = games.loc[((games.home_club_id == club_home_id) | (games.away_club_id == club_home_id))\
        & ((games.home_club_id == club_away_id) | (games.away_club_id == club_away_id))\
        & (games.date < target_game['date'].values[0]) ,:].sort_values('date',ascending=False)[:N]
    
    if N_last_games.shape[0]==0:
        return None

    else:
        N_last_games[['H_goals','H_goals_conceded','H_HorA']] = N_last_games.apply(lambda x: \
        pd.Series([x['home_club_goals'], x['away_club_goals'],0]) if x['home_club_id']==club_home_id else pd.Series([x['away_club_goals'], x['home_club_goals'],1])\
        , axis = 1)

        H_cards = appearance.loc[(appearance.game_id.isin(N_last_games.game_id.values)) \
                & (appearance.player_club_id == club_home_id),\
                    ['game_id','yellow_cards','red_cards']].groupby('game_id').sum().reset_index()\
                        .rename(columns={'yellow_cards': 'H_yellow_cards' ,'red_cards' : 'H_red_cards'})

        A_cards = appearance.loc[(appearance.game_id.isin(N_last_games.game_id.values)) \
                & (appearance.player_club_id == club_away_id),\
                    ['game_id','yellow_cards','red_cards']].groupby('game_id').sum().reset_index()\
                        .rename(columns={'yellow_cards': 'A_yellow_cards' ,'red_cards' : 'A_red_cards'})

        N_last_games.drop(columns = ['date','home_club_id','away_club_id','home_club_goals','away_club_goals'],inplace = True)
        res = pd.merge(N_last_games, H_cards, on='game_id')
        res = pd.merge(res, A_cards, on='game_id')
        res.game_id= game_id
        return res

In [172]:
game_history = game_ids.progress_apply(get_game_history)
game_history = pd.concat(game_history.values)
game_history = game_history.reset_index(drop = True).reset_index().rename(columns={'index':'game_history_id'})


100%|██████████| 29545/29545 [26:13<00:00, 18.78it/s]  


In [175]:
len(game_history.game_id.unique())

25136

In [188]:
game_history.groupby('game_id')[['game_history_id']].count().value_counts()

game_history_id
5                  12875
1                   4242
2                   2942
3                   2755
4                   2322
dtype: int64

In [176]:
len(train_df.game_id.unique())

29545

In [178]:
game_history.game_id.unique() 

array([2459642, 2459657, 2459672, ..., 3700967, 3700970, 3710372])

In [189]:
game_history.to_csv('data/train_LSTM/game_history_full.csv')